In [1]:
import findspark 
findspark.init('/home/abraham/spark-2.2.1-bin-hadoop2.7') 
import pyspark
import os
import sys
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext()
sqlContext = SQLContext(sc) 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('cruise').getOrCreate() 
flights = spark.read.csv('/home/abraham/MGE_2018/tarea4/flights.csv',inferSchema=True,header=True)

In [28]:
flights

DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]

In [2]:
flights=flights.na.fill(0) #imputamos los missings de las variables numércias
flights=flights.na.fill("?")#imputamos los missng de las variables categóricas

In [3]:
#Seleccionamos la lista de variables de tipo string y las guardamos en la variable columnList
columnList = [item[0] for item in flights.dtypes if item[1].startswith('string')]

In [4]:
#Hacemos la partición de datos 70% para entrenamiento y 30 para test.
train_data,test_data = flights.randomSplit([0.7,0.3])

In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector
#Creamos la función indexers para transformar los features de tipo categóricos a numéricos
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(flights) for column in 
            list(set(columnList)) ]

In [6]:
from pyspark.ml import Pipeline
#Creamos un pipeline pindexers para que la función indexers tenga métodos fit transform.
pindexers = Pipeline(stages=indexers) 

In [29]:
#Hacemos VectorAssembler para poder generar la variable features que es un vector de los valores de cada columna
#en este caso eliminamos algunas columnas ya que no hace sentido ocuparlas tipo el año, y variables de horas.
assembler = VectorAssembler(
  inputCols=[#'YEAR',Thin
             'MONTH',
             'DAY',
             'DAY_OF_WEEK',
             #'AIRLINE',
            'AIRLINE_index',
             'FLIGHT_NUMBER',
            # 'TAIL_NUMBER',
            #'ORIGIN_AIRPORT',
            'ORIGIN_AIRPORT_index',
            #'DESTINATION_AIRPORT',
            'DESTINATION_AIRPORT_index',
            'SCHEDULED_DEPARTURE',
            #'DEPARTURE_TIME',
            'TAXI_OUT',
            #'WHEELS_OFF',
            #'SCHEDULED_TIME',
            'ELAPSED_TIME',
            'AIR_TIME',
            'DISTANCE',
            #'WHEELS_ON',
            'TAXI_IN',
            #'SCHEDULED_ARRIVAL',
            #'ARRIVAL_TIME',
            'ARRIVAL_DELAY',
            #'DIVERTED',
            #'CANCELLED',
            #'CANCELLATION_REASON',
            #'AIR_SYSTEM_DELAY',
            #'SECURITY_DELAY',
            #'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'
  ],
    outputCol="features")

In [9]:
#Normalizamos los features
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features") 

In [10]:
#aplicamos pca para reducir dimensionalidad
pca = PCA(k=7, inputCol="scaled_features", outputCol="pcaFeatures")

In [15]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.regression import LinearRegression

In [16]:
#Hacemos un udf para generar un tipo diccionario de modelos y de Paramgridbuilders rspectivamente y estos
#se iteren con sus respectivos paraétros.
def define_hyper_params():

    #Creamos un diccionario de los modelos
    modelo = {'lr': LinearRegression(featuresCol="pcaFeatures",labelCol='DEPARTURE_DELAY'),
    'rf': RandomForestRegressor(featuresCol="pcaFeatures",labelCol="DEPARTURE_DELAY")}
    
    #Creamos una lista de paramgrids para tener la lista de prarámetros con los que se hará Cross validtion.
    #en este caso para lr y rf.
    search_space = [ParamGridBuilder().\
    addGrid(modelo['lr'].regParam, [0,0.01,0.05,0.1,.2]).\
    addGrid(modelo['lr'].elasticNetParam, [0,.01,.05,.1,.2]).\
    build()
    ,
    ParamGridBuilder().\
    addGrid(modelo['rf'].numTrees, [10,20]).\
    addGrid(modelo['rf'].maxDepth, [5, 10]).\
    build()]

    return (modelo,search_space)

In [17]:
from pyspark.ml import Pipeline
def magic_loop(X_train,models_to_run=['lr','rf']):#entradas dataframe X_train y diccionario de modelos
    modelo,search_space=define_hyper_params() #usamos la función define_hyper_params para seleccionar 
    #los paramgrids definidos.
    best=[] #lista para guardar los mejores modelos de cada algoritmo
    metr=[] #lista para guardar las métricas de ls mejores mdelos de cada algoritmo
    params=[] #lista para guardar los mejores parámetros de cada algoritmo.
    for i in range(len(models_to_run)):#corremos para cada modelo sus respectivos parametros
        #generamos el pipeline de todos los transformers que declaramos
        pipeline = Pipeline(stages=[pindexers,assembler,scaler,pca,modelo[models_to_run[i]]]) 
        #hacemos el cross validation con la lista de paramétros del models_to_run[i]
        crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=search_space[i],#parametros del modelo i
                          evaluator=RegressionEvaluator(predictionCol='prediction', labelCol="DEPARTURE_DELAY", 
                               metricName='rmse'),#metrica de comparacion default es rmse
                          numFolds=2)#corremos con 10 partciones el cross validation
        cvModel = crossval.fit(X_train)#ajuste del modelo
        best_model=cvModel.bestModel#generamos el mejor modelo con los mejores parametros para el algoritmo i
        best.append(best_model) #guardamos el mejor modelo del algoritmo i en la lista vest
        metr.append(min(cvModel.avgMetrics)) #guardamos la metrica rmse en la lista metr del mejor modelo
        #lo mismo con los parametros
        params.append(search_space[i][cvModel.avgMetrics.index(min(cvModel.avgMetrics))])
        
        #imprimimos los parametros del mejor modelo del algoritmo i
        print('Mejor modelo de ',models_to_run[i],'fue:',search_space[i][cvModel.avgMetrics.index(min(cvModel.avgMetrics))]) 
    
    print(metr)
    
    #imprimimos el modelo ganador
    print('Mejor modelo fue:',params[metr.index(min(metr))]) 
    
    #El mejor modelo es el que tienen el menor rmse y eso regresa el programa.        
    return(best[metr.index(min(metr))])

In [56]:
%%time 
magic_loop(train_data,models_to_run=['lr','rf'])

Mejor modelo de  lr fue: {Param(parent='LinearRegression_481b82250acf6fe8d39e', name='regParam', doc='regularization parameter (>= 0).'): 0, Param(parent='LinearRegression_481b82250acf6fe8d39e', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0}
Mejor modelo de  rf fue: {Param(parent='RandomForestRegressor_45b5a0b3dff6e079312f', name='numTrees', doc='Number of trees to train (>= 1).'): 20}
[36.51568244674939, 36.24954170922881]
Mejor modelo fue: {Param(parent='RandomForestRegressor_45b5a0b3dff6e079312f', name='numTrees', doc='Number of trees to train (>= 1).'): 20}


PipelineModel_48bea2dd173a8762eefc

In [35]:
%%time
#guardamos el mejor modelo lo guardamos en champ y tomamos el tiempo de ejecucion de magic loop
champ=magic_loop(train_data,models_to_run=['lr','rf'])

Mejor modelo de  lr fue: {Param(parent='LinearRegression_48ff93cf0a23b4c74f5b', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0, Param(parent='LinearRegression_48ff93cf0a23b4c74f5b', name='regParam', doc='regularization parameter (>= 0).'): 0.01}
Mejor modelo de  rf fue: {Param(parent='RandomForestRegressor_4c46965d7d6e12ef7014', name='numTrees', doc='Number of trees to train (>= 1).'): 20, Param(parent='RandomForestRegressor_4c46965d7d6e12ef7014', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 10}
[36.498729960099396, 35.99097917344129]
Mejor modelo fue: {Param(parent='RandomForestRegressor_4c46965d7d6e12ef7014', name='numTrees', doc='Number of trees to train (>= 1).'): 20, Param(parent='RandomForestRegressor_4c46965d7d6e12ef7014', name='maxDepth', doc='Maximum depth of the tree. 

In [17]:
train_data.count()

4073924

In [36]:
#Ajustamos el modelo ganador a los datos de test.
Results=champ.transform(test_data)

In [37]:
Results.select("DEPARTURE_DELAY",'prediction').show()

+---------------+------------------+
|DEPARTURE_DELAY|        prediction|
+---------------+------------------+
|            108| 18.64875623400596|
|            -11|  6.28328632994873|
|             -6|18.552537589269697|
|             -4|3.1351813280716385|
|              3|19.105606553924364|
|             -6|6.2470002388006005|
|             -3|17.141745772777128|
|             -4|19.105606553924364|
|             23| 7.492240299219928|
|              9|18.896973779743256|
|             -7| 7.055545795550583|
|              3|18.900372410219646|
|              0| 7.799763384514726|
|             -5| 8.110033905954362|
|             -2| 7.638137819692622|
|            121| 19.26445126012523|
|              4|4.9338790903378955|
|              5|18.552537589269697|
|             16|12.296915427067606|
|              1|18.776194097934486|
+---------------+------------------+
only showing top 20 rows



In [2]:
import datetime
a=datetime.datetime.now()

b=datetime.datetime.now()
print('tiempo de proceso',b-a)

tiempo de proceso 0:00:00.000144
